# RF-DETR Fine-tuning on Google Colab

This notebook demonstrates how to fine-tune RF-DETR for drone detection using GPU acceleration on Google Colab.

## Prerequisites
1. Enable GPU runtime: Runtime > Change runtime type > Hardware accelerator > GPU
2. Upload your dataset to Google Drive or prepare it for upload to Colab

In [ ]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name()}")
    print(f"Device memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ GPU not available. Please enable GPU runtime: Runtime > Change runtime type > Hardware accelerator > GPU")

In [ ]:
# Install required packages
!pip install rfdetr==1.2.1
!pip install supervision==0.26.1
!pip install roboflow
!pip install tensorboard

In [ ]:
# Mount Google Drive (optional - if your dataset is stored there)
from google.colab import drive
drive.mount('/content/drive')

# If dataset is in Google Drive, set the path accordingly
# dataset_location = "/content/drive/MyDrive/path/to/your/dataset"

In [ ]:
# Option 1: Upload dataset as zip file
# Uncomment and run this cell if you want to upload a dataset zip file

# from google.colab import files
# import zipfile
# import os

# print("Please upload your dataset zip file...")
# uploaded = files.upload()

# # Extract the uploaded zip file
# for filename in uploaded.keys():
#     if filename.endswith('.zip'):
#         with zipfile.ZipFile(filename, 'r') as zip_ref:
#             zip_ref.extractall('/content/')
#         print(f"Extracted {filename}")
#         # Remove the zip file to save space
#         os.remove(filename)

# dataset_location = "/content/dataset"  # Adjust path as needed

In [ ]:
# Option 2: Download dataset from URL (if available)
# Uncomment and modify this cell if your dataset is available via URL

# import requests
# import zipfile
# from io import BytesIO

# dataset_url = "YOUR_DATASET_URL_HERE"  # Replace with your dataset URL
# response = requests.get(dataset_url)
# with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
#     zip_ref.extractall('/content/')

# dataset_location = "/content/dataset"  # Adjust path as needed

In [ ]:
# Option 3: Download from Roboflow (if your dataset is hosted there)
# Uncomment and modify this cell if using Roboflow

# import roboflow
# from roboflow import Roboflow

# rf = Roboflow(api_key="YOUR_API_KEY")  # Replace with your API key
# project = rf.workspace("YOUR_WORKSPACE").project("YOUR_PROJECT")  # Replace with your details
# dataset = project.version(1).download("coco", location="/content")

# dataset_location = dataset.location

In [ ]:
# Set dataset location manually if using any of the above methods
# Adjust this path based on where your dataset was extracted/downloaded
dataset_location = "./dataset"  # Change this to match your dataset location

# Verify dataset structure
import os
if os.path.exists(dataset_location):
    print(f"Dataset found at: {dataset_location}")
    print("Dataset structure:")
    for root, dirs, files in os.walk(dataset_location):
        level = root.replace(dataset_location, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files[:5]:  # Show first 5 files only
            print(f"{subindent}{file}")
        if len(files) > 5:
            print(f"{subindent}... and {len(files)-5} more files")
else:
    print(f"⚠️ Dataset not found at {dataset_location}")
    print("Please upload your dataset using one of the methods above.")

In [ ]:
# Import RF-DETR
from rfdetr import RFDETRMedium

In [ ]:
# Initialize and train the model
# The model will automatically use GPU if available
model = RFDETRMedium()

# Training parameters optimized for Colab
# Adjust batch_size based on your GPU memory
# For Colab's free T4 GPU, batch_size=4-8 should work well
model.train(
    dataset_dir=dataset_location, 
    epochs=10,              # Increased epochs for better training
    batch_size=4,           # Reduced for better GPU memory usage
    grad_accum_steps=4,     # Increased to simulate larger batch size
    lr=1e-3,                # Slightly reduced learning rate
    device="cuda" if torch.cuda.is_available() else "cpu"  # Explicitly set device
)

In [ ]:
# Save the trained model
model.save_model("rf_detr_drone_detection.pth")
print("Model saved as 'rf_detr_drone_detection.pth'")

# Download the model to your local machine
from google.colab import files
files.download("rf_detr_drone_detection.pth")

In [ ]:
# Load test dataset for evaluation
import supervision as sv

ds = sv.DetectionDataset.from_coco(
    images_directory_path=f"{dataset_location}/test",
    annotations_path=f"{dataset_location}/test/_annotations.coco.json",
)

In [ ]:
# Optimize model for inference
model.optimize_for_inference()

In [ ]:
# Visualize predictions vs ground truth
from PIL import Image
import matplotlib.pyplot as plt

images = []
titles = []

# Reduce number of images for faster processing in Colab
num_samples = min(5, len(ds))  # Show 5 samples or less if dataset is smaller

for i in range(num_samples):
    path, image, annotations = ds[i]
    image = Image.open(path)
    
    # Get predictions
    detections = model.predict(image, threshold=0.5)

    # Calculate optimal visualization parameters
    text_scale = sv.calculate_optimal_text_scale(resolution_wh=image.size)
    thickness = sv.calculate_optimal_line_thickness(resolution_wh=image.size)
    color = sv.ColorPalette.from_hex([
        "#ffff00", "#ff9b00", "#ff66ff", "#3399ff", "#ff66b2", "#ff8080",
        "#b266ff", "#9999ff", "#66ffff", "#33ff99", "#66ff66", "#99ff00"
    ])

    # Create annotators
    bbox_annotator = sv.BoxAnnotator(color=color, thickness=thickness)
    label_annotator = sv.LabelAnnotator(
        color=color,
        text_color=sv.Color.BLACK,
        text_scale=text_scale
    )

    # Create labels
    annotations_labels = [
        f"{ds.classes[class_id]}"
        for class_id in annotations.class_id
    ]

    detections_labels = [
        f"{ds.classes[class_id]} {confidence:.2f}"
        for class_id, confidence
        in zip(detections.class_id, detections.confidence)
    ]

    # Annotate ground truth
    annotation_image = image.copy()
    annotation_image = bbox_annotator.annotate(annotation_image, annotations)
    annotation_image = label_annotator.annotate(annotation_image, annotations, annotations_labels)

    # Annotate predictions
    detections_image = image.copy()
    detections_image = bbox_annotator.annotate(detections_image, detections)
    detections_image = label_annotator.annotate(detections_image, detections, detections_labels)

    images.extend([annotation_image, detections_image])
    titles.extend([f"Ground Truth {i+1}", f"Prediction {i+1}"])

# Plot results
sv.plot_images_grid(images=images, grid_size=(num_samples, 2), titles=titles)

In [ ]:
# Evaluate model performance (optional)
# This cell provides basic evaluation metrics

from supervision import MeanAveragePrecision

# Collect all predictions and ground truths
all_predictions = []
all_targets = []

print("Evaluating model on test set...")
for i in range(len(ds)):
    path, image, annotations = ds[i]
    image = Image.open(path)
    
    # Get predictions
    detections = model.predict(image, threshold=0.3)  # Lower threshold for evaluation
    
    all_predictions.append(detections)
    all_targets.append(annotations)
    
    if (i + 1) % 10 == 0:
        print(f"Processed {i + 1}/{len(ds)} images")

# Calculate mAP
map_metric = MeanAveragePrecision()
map_result = map_metric.update(all_predictions, all_targets).compute()

print(f"\nEvaluation Results:")
print(f"mAP@0.5: {map_result.map50:.3f}")
print(f"mAP@0.5:0.95: {map_result.map50_95:.3f}")

In [ ]:
# View TensorBoard logs (if available)
# Uncomment if RF-DETR generates TensorBoard logs

# %load_ext tensorboard
# %tensorboard --logdir ./output

## Next Steps

1. **Model Performance**: If the model performance is not satisfactory, consider:
   - Increasing the number of epochs
   - Adjusting learning rate
   - Data augmentation
   - Fine-tuning hyperparameters

2. **Deployment**: The trained model can be:
   - Downloaded and used locally
   - Deployed to cloud services
   - Integrated into applications

3. **Further Training**: You can continue training by loading the saved model:
   ```python
   model = RFDETRMedium.load_model("rf_detr_drone_detection.pth")
   model.train(dataset_dir=dataset_location, epochs=5)  # Continue training
   ```